In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import time
import pandas as pd

In [2]:
# Create an empty dataframe to hold the salary data

In [3]:
salary_df = pd.DataFrame()

In [4]:
# Open a web browser with Selenium and browser to the state salary website

In [5]:
driver = webdriver.Chrome()
driver.get("http://www.in.gov/apps/gov/salaries/search")
time.sleep(5)

In [6]:
# Choose an agency from the dropdown list and search

In [7]:
agency = Select(driver.find_element_by_name('agency'))
agency.select_by_visible_text("INSURANCE") # Change this value to the agency for which you're retrieving salary data 

In [8]:
search = driver.find_element_by_xpath('//*[@id="searchTable"]/tbody/tr[4]/td[2]/input')
search.click()
time.sleep(5)

In [9]:
# Parse the HTML with BeautifulSoup and append values to the dataframe

In [10]:
while True:
    try:
        html_source = driver.page_source
        salaryinfo = BeautifulSoup(html_source, 'lxml')
        salary_table = salaryinfo.findAll("table")
        salary_rows = salary_table[1].findAll(lambda tag: tag.name=='tr')
        salary_rows.pop(0)

        for i in salary_rows:
            employee = i.findAll('td')
            firstname = employee[0].text
            lastname = employee[1].text
            agency = employee[2].text
            status = employee[3].text
            salary = employee[4].text

            salary_df = salary_df.append({'FirstName': firstname,
                                            'LastName': lastname,
                                            'Agency': agency,
                                            'Status': status,
                                            'Salary': salary}, ignore_index=True)
        nextlist = driver.find_element_by_class_name('nextLink')
        nextlist.click()
        time.sleep(5)
    except Exception as e:
        if "Unable to locate element" in str(e):
            salary_df.to_csv('salaries.csv', index=False)
            print("Wrote data to salaries.csv")
        else:
            print("Error")
        break

Wrote data to salaries.csv
